# Finding the next NBA City

### Introduction

The sports entertainment industry is a multi-billion dollar industry. Comprised of four major sports leagues (the NFL, the NBA, MLB and the NHL) each with roughly 30 teams, the impact of these leagues is felt around the country. Yet, in spite of the seeming national saturation of sports teams and leagues, there is continual talk of adding additional teams to new cities to further expand their respective sports. 

A league that has seen growing popularity as of late is the National Basketball Association (NBA). In light of this, talk has been swirling of adding an expansion team to a new market. To further fuel these rumors, the NBA has relatively fewer teams than its two Fall/Winter sports league rivals; 30 teams vs 32 for the  NFL and (soon to be) 32 for the NHL.

With this as a backdrop, the NBA is looking to expand its team base and, therefore, fan and revenue base via expansion. The challenge is finding a city that is suitable for expansion and will create a corresponding increase in revenue without diluting the current product. 

The goal of this study is to help the NBA narrow down its search for suitable expansion cities. This will be done by looking at both bulk city data (population, TV market size, income) and the interests of the locals via popular venues and locales within the respective cities. 

In [189]:
# make the imports
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import sys
#!{sys.executable} -m pip install lxml
#import lxml

### Data

In addition to the FourSqure locale data, there are 4 data sources we need:

1. City Population Data - https://en.m.wikipedia.org/wiki/List_of_United_States_cities_by_population
2. TV Market Data - https://en.wikipedia.org/wiki/List_of_United_States_television_markets
3. Metro Area income data - https://en.wikipedia.org/wiki/List_of_United_States_metropolitan_areas_by_per_capita_income
4. List of current NBA teams - https://www.basketball-reference.com/teams/ 

The first three are located in Wikipedia and will need to be scraped separately then merged for later use.

The last one was exported as a .csv file through the basketball-reference website and will be read-in to a pandas dataframe directly

#### Population Data

First, we'll pull in the population data from Wikipedia

In [266]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [537]:
pop_wiki = requests.get("https://en.m.wikipedia.org/wiki/List_of_United_States_cities_by_population")
pop_soup = BeautifulSoup(pop_wiki.content,'html')
#pop_soup

/usr/lib/python3/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [538]:
# get the table reference
# By inspection of HTML, table we are interested in is stored in index 3
table = pop_soup.find_all('table')[3]
#table

We will define a function for extracting the table into a dataframe for the population data wiki page

In [539]:
def get_table_data(table):
    
    # Extract the column names
    columns = []
    for c in table.find_all('th'):
        columns.append(c.get_text().strip())
   
    # get the remaining elements based on the length of the column headers
    num_cols = len(columns)
    # create an empty dataframe with the columns discovered
    df = pd.DataFrame()
    
    i=0
    row=[]
    
    for td in table.find_all('td'):
        #print(td.get_text())
        if ('sq' in td.get_text()) and ('mi' in td.get_text()):
            i=i-1
        else:
            row.append(td.get_text().strip())
        i+=1
        # once the number of elements in the rows equals the number of columns, add it to the df and reset the row
        if i==num_cols:
            i=0
            df = df.append(pd.DataFrame(row).T)
            row=[]
    df.columns=columns
    return df
   

In [540]:
df = get_table_data(table)
cols = df.columns.values
cols[0]='Rank'
df.columns = cols
df.set_index('Rank', inplace=True) 
df

,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 population density,Location
Rank,,,,,,,,
1,New York[d],New York,"8,398,748","8,175,133",+2.74%,780.9 km2,"10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,"1,213.9 km2","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,588.7 km2,"4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,"1,651.1 km2","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,"1,340.6 km2","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...
6,Philadelphia[e],Pennsylvania,"1,584,138","1,526,006",+3.81%,347.6 km2,"4,511/km2",40°00′34″N 75°08′00″W﻿ / ﻿40.0094°N 75.1333°W﻿...
7,San Antonio,Texas,"1,532,233","1,327,407",+15.43%,"1,194.0 km2","1,250/km2",29°28′21″N 98°31′30″W﻿ / ﻿29.4724°N 98.5251°W﻿...
8,San Diego,California,"1,425,976","1,307,402",+9.07%,842.3 km2,"1,670/km2",32°48′55″N 117°08′06″W﻿ / ﻿32.8153°N 117.1350°...
9,Dallas,Texas,"1,345,047","1,197,816",+12.29%,882.9 km2,"1,493/km2",32°47′36″N 96°45′59″W﻿ / ﻿32.7933°N 96.7665°W﻿...


Now we need to simplify this data to only keep what we need (City, State, 2018 estimated population, 2016 population density)

In [541]:
df_pop = df[ ['City','State[c]','2018estimate','2016 population density'] ]
# rename the columns
df_pop.columns = ['City','State','Population','Density']
df_pop

,City,State,Population,Density
Rank,,,,
1,New York[d],New York,"8,398,748","10,933/km2"
2,Los Angeles,California,"3,990,456","3,276/km2"
3,Chicago,Illinois,"2,705,994","4,600/km2"
4,Houston[3],Texas,"2,325,502","1,395/km2"
5,Phoenix,Arizona,"1,660,272","1,200/km2"
6,Philadelphia[e],Pennsylvania,"1,584,138","4,511/km2"
7,San Antonio,Texas,"1,532,233","1,250/km2"
8,San Diego,California,"1,425,976","1,670/km2"
9,Dallas,Texas,"1,345,047","1,493/km2"


Lastly, we will remove the /km2 from the density column and convert to type float

In [542]:
df_pop['Density']=df_pop['Density'].replace('/km2','',regex=True)
df_pop['City']=df_pop['City'].replace('\[.*\]','',regex=True)
df_pop

/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,City,State,Population,Density
Rank,,,,
1,New York,New York,"8,398,748","10,933"
2,Los Angeles,California,"3,990,456","3,276"
3,Chicago,Illinois,"2,705,994","4,600"
4,Houston,Texas,"2,325,502","1,395"
5,Phoenix,Arizona,"1,660,272","1,200"
6,Philadelphia,Pennsylvania,"1,584,138","4,511"
7,San Antonio,Texas,"1,532,233","1,250"
8,San Diego,California,"1,425,976","1,670"
9,Dallas,Texas,"1,345,047","1,493"


#### Media Market Data

Next, we'll pull in the tv media market data from Wikipedia

In [543]:
tv_wiki = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_television_markets")
tv_soup = BeautifulSoup(tv_wiki.content,'html')
tv_soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en"><head>
<meta charset="utf-8"/>
<title>List of United States television markets - Wikipedia</title>
<script>document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_United_States_television_markets","wgTitle":"List of United States television markets","wgCurRevisionId":914065393,"wgRevisionId":914065393,"wgArticleId":44933675,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles to be merged from January 2019","All articles to be merged","Market (economics)","American television-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","Janua

In [544]:
# get the table reference
# By inspection of HTML, table we are interested in is stored in index 1
table = tv_soup.find_all('table')[1]
#table

In [545]:
def get_tv_table_data(table):
    
    # Extract the column names
    columns = []
    for c in table.find_all('th'):
        if 'scope' in c.attrs:
            pass
        else:
            columns.append(c.get_text().strip())
   
    # get the remaining elements based on the length of the column headers
    num_cols = len(columns)
    # create an empty dataframe with the columns discovered
    df = pd.DataFrame()
    
    i=0
    row=[]
    
    # get the market column based on the th tag
    markets = []
    for market in table.find_all('th'):
        if 'scope' in market.attrs:
            markets.append(market.get_text().strip())
   
    # get the remaining elements based on the length of the column headers
    num_cols = len(columns)
    # create an empty dataframe with the columns discovered
    df = pd.DataFrame()
    
    i=0
    j=0
    row=[]
       
    for td in table.find_all('td'):
        if i==1:
            try:
                row.append(markets[j])
                j+=1
                i+=1
            except:
                pass

        # if it's a name with a link, get the name out of the link otherwise just append it to the row
        if td.find('a')!=None:
            row.append(td.find('a').get_text())
        else:
            row.append(td.get_text().strip())
        i+=1
        # once the number of elements in the rows equals the number of columns, add it to the df and reset the row
        if i==num_cols:
            i=0
            df = df.append(pd.DataFrame(row).T)
            row=[]

    df.columns=columns
    df.set_index('Rank[1]', inplace=True)
    return df

In [546]:
df_tv = get_tv_table_data(table)
df_tv.head()

,Market,State,Counties (or county-equivalents) covered,TV households (2018–19),Local ABC affiliate,Local CBS affiliate,Local CW affiliate,Local Fox affiliate,Local NBC affiliate,Other significant stations[2]
Rank[1],,,,,,,,,,
1,New York,New York,Bronx,"7,100,300 (6.441%)",WABC-TV,WCBS-TV,WPIX,WNYW,WNBC,WFTY-DT
2,Los Angeles,California,Inyo,"5,276,600 (4.786%)",KABC-TV,KCBS-TV,KTLA,KTTV,KNBC,KCAL-TV
3,Chicago,Illinois,Cook,"3,251,370 (2.949%)",WLS-TV,WBBM-TV,WCIU-TV,WFLD,WMAQ-TV,WGN-TV
4,Philadelphia,Pennsylvania,Berks,"2,816,850 (2.555%)",WPVI-TV,KYW-TV,WPSG,WTXF-TV,WCAU,WFPA-CD
5,Dallas-Fort Worth,Texas,Anderson,"2,622,070 (2.378%)",WFAA,KTVT,KDAF,KDFW,KXAS-TV,KDFI


Now we'll remove the extraneous information and keep only what we need

In [547]:
df_tv_market = df_tv[ ['Market', 'State', 'TV households (2018–19)']]
df_tv_market

,Market,State,TV households (2018–19)
Rank[1],,,
1,New York,New York,"7,100,300 (6.441%)"
2,Los Angeles,California,"5,276,600 (4.786%)"
3,Chicago,Illinois,"3,251,370 (2.949%)"
4,Philadelphia,Pennsylvania,"2,816,850 (2.555%)"
5,Dallas-Fort Worth,Texas,"2,622,070 (2.378%)"
6,Washington (Hagerstown),District of Columbia,"2,482,480 (2.252%)"
7,Houston,Texas,"2,423,360 (2.198%)"
8,San Francisco-Oakland-San Jose,California,"2,414,470 (2.19%)"
9,Boston (Manchester),Massachusetts,"2,364,870 (2.145%)"


#### Median Income Data

Next, we'll pull in the median income data

In [548]:
inc_wiki = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_metropolitan_areas_by_per_capita_income")
inc_soup = BeautifulSoup(inc_wiki.content,'html')
#inc_soup

In [549]:
# get the table reference
# By inspection of HTML, table we are interested in is stored in index 2
table = inc_soup.find_all('table')[2]
#table

In [550]:
df_inc = get_table_data(table)
df_inc.set_index('Rank', inplace=True)
df_inc.rename(columns={'Metropolitan statistical area':'Metro'}, inplace=True)
df_inc.drop('Population', axis=1,inplace=True)
df_inc

,Metro,Per capitaincome
Rank,,
1,"Washington-Arlington-Alexandria, D.C-Virginia-...","$47,411"
2,"San Jose-Santa Clara-Sunnyvale, California MSA","$40,392"
3,"Seattle-Tacoma-Bellevue, Washington MSA","$39,322"
4,"San Francisco-Oakland-Hayward, California MSA","$38,355"
5,"Boston–Worcester–Lawrence, Massachusetts–New H...","$37,311"
6,"Honolulu, Hawaii MSA","$36,339"
7,"Minneapolis-St. Paul-Bloomington, Minnesota MSA","$35,388"
8,"Hartford, Connecticut MSA","$34,310"
9,"Denver-Aurora-Lakewood, Colorado MSA","$32,399"


### NBA Team Cities
Finally, we'll pull in the list of NBA team cities by CSV

In [551]:
df_teams = pd.read_csv('NBA_cities.csv')

# filter by current teams (where To = 2020)
df_teams = df_teams[ df_teams['To']==2020 ]
# get list of unique NBA cities
nba_cities = list(df_teams.Franchise.unique())
nba_cities

['Atlanta Hawks',
 'Boston Celtics',
 'Brooklyn Nets',
 'Charlotte Hornets',
 'Chicago Bulls',
 'Cleveland Cavaliers',
 'Dallas Mavericks',
 'Denver Nuggets',
 'Detroit Pistons',
 'Golden State Warriors',
 'Houston Rockets',
 'Indiana Pacers',
 'Los Angeles Clippers',
 'Los Angeles Lakers',
 'Memphis Grizzlies',
 'Miami Heat',
 'Milwaukee Bucks',
 'Minnesota Timberwolves',
 'New Orleans Pelicans',
 'New York Knicks',
 'Oklahoma City Thunder',
 'Orlando Magic',
 'Philadelphia 76ers',
 'Phoenix Suns',
 'Portland Trail Blazers',
 'Sacramento Kings',
 'San Antonio Spurs',
 'Toronto Raptors',
 'Utah Jazz',
 'Washington Wizards']

### Merge Data Sets for Use
Now we will merge all of the datasets into a usable dataframe

First we'll add the per capita income

In [552]:
# store metro and pci information
metros = list(df_inc.Metro)
pci = list(df_inc['Per capitaincome'])

# create a function to extract it
def get_merged_value(city,options,values):
    for i in range(len(options)):
        if city in options[i]:
            return values[i]
    return np.nan
            

df_pop['PCI'] = df_pop['City'].apply(lambda x: get_merged_value(x,metros,pci))
df_pop

/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,City,State,Population,Density,PCI
Rank,,,,,
1,New York,New York,"8,398,748","10,933","$24,581"
2,Los Angeles,California,"3,990,456","3,276","$21,170"
3,Chicago,Illinois,"2,705,994","4,600",NaN
4,Houston,Texas,"2,325,502","1,395","$21,701"
5,Phoenix,Arizona,"1,660,272","1,200","$21,907"
6,Philadelphia,Pennsylvania,"1,584,138","4,511","$22,874"
7,San Antonio,Texas,"1,532,233","1,250","$18,518"
8,San Diego,California,"1,425,976","1,670","$22,926"
9,Dallas,Texas,"1,345,047","1,493","$23,616"


In [553]:
# casting as type float
df_pop['Population'] = df_pop['Population'].str.replace(',','').astype('float')
df_pop['Density'] = df_pop['Density'].str.replace(',','').astype('float')
df_pop['PCI'] = df_pop['PCI'].str.replace(',','').str.replace('$','').astype('float')
# fixing the Nan, filling by state
df_data = df_pop.interpolate()
df_data.index = df_data.index.astype(int)
df_data.sort_index(inplace=True)
df_data

/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

,City,State,Population,Density,PCI
Rank,,,,,
1,New York,New York,8398748.0,10933.0,24581.000000
2,Los Angeles,California,3990456.0,3276.0,21170.000000
3,Chicago,Illinois,2705994.0,4600.0,21435.500000
4,Houston,Texas,2325502.0,1395.0,21701.000000
5,Phoenix,Arizona,1660272.0,1200.0,21907.000000
6,Philadelphia,Pennsylvania,1584138.0,4511.0,22874.000000
7,San Antonio,Texas,1532233.0,1250.0,18518.000000
8,San Diego,California,1425976.0,1670.0,22926.000000
9,Dallas,Texas,1345047.0,1493.0,23616.000000


In [554]:
df_pop.dtypes

City           object
State          object
Population    float64
Density       float64
PCI           float64
dtype: object

Adding the tv market in a similar fashion

In [555]:
market = list(df_tv_market['Market'])
households = list(df_tv_market['TV households (2018–19)'])

df_data['TV'] = df_data['City'].apply(lambda x: get_merged_value(x,market,households))
df_data

,City,State,Population,Density,PCI,TV
Rank,,,,,,
1,New York,New York,8398748.0,10933.0,24581.000000,"7,100,300 (6.441%)"
2,Los Angeles,California,3990456.0,3276.0,21170.000000,"5,276,600 (4.786%)"
3,Chicago,Illinois,2705994.0,4600.0,21435.500000,"3,251,370 (2.949%)"
4,Houston,Texas,2325502.0,1395.0,21701.000000,"2,423,360 (2.198%)"
5,Phoenix,Arizona,1660272.0,1200.0,21907.000000,"1,864,420 (1.691%)"
6,Philadelphia,Pennsylvania,1584138.0,4511.0,22874.000000,"2,816,850 (2.555%)"
7,San Antonio,Texas,1532233.0,1250.0,18518.000000,"923,990 (0.838%)"
8,San Diego,California,1425976.0,1670.0,22926.000000,"987,760 (0.896%)"
9,Dallas,Texas,1345047.0,1493.0,23616.000000,"2,622,070 (2.378%)"


In [556]:
# fix the NaN by interpolation after sorting by population
df_data['TV'] = df_data['TV'].str.replace('\(.*\)','').str.replace(',','').astype('float')
# fixing the Nan, filling by state
df_data = df_data.interpolate()
df_data.sort_index(inplace=True)
df_data

,City,State,Population,Density,PCI,TV
Rank,,,,,,
1,New York,New York,8398748.0,10933.0,24581.000000,7.100300e+06
2,Los Angeles,California,3990456.0,3276.0,21170.000000,5.276600e+06
3,Chicago,Illinois,2705994.0,4600.0,21435.500000,3.251370e+06
4,Houston,Texas,2325502.0,1395.0,21701.000000,2.423360e+06
5,Phoenix,Arizona,1660272.0,1200.0,21907.000000,1.864420e+06
6,Philadelphia,Pennsylvania,1584138.0,4511.0,22874.000000,2.816850e+06
7,San Antonio,Texas,1532233.0,1250.0,18518.000000,9.239900e+05
8,San Diego,California,1425976.0,1670.0,22926.000000,9.877600e+05
9,Dallas,Texas,1345047.0,1493.0,23616.000000,2.622070e+06


Finally, determine if the city already has an NBA team

In [557]:
# define a function to add it
def nba_city(city,nba_cities):
    for n in nba_cities:
        if city in n:
            return 1
    return 0

df_data['NBA']=df_data['City'].apply(lambda x: nba_city(x,nba_cities))
df_data

,City,State,Population,Density,PCI,TV,NBA
Rank,,,,,,,
1,New York,New York,8398748.0,10933.0,24581.000000,7.100300e+06,1
2,Los Angeles,California,3990456.0,3276.0,21170.000000,5.276600e+06,1
3,Chicago,Illinois,2705994.0,4600.0,21435.500000,3.251370e+06,1
4,Houston,Texas,2325502.0,1395.0,21701.000000,2.423360e+06,1
5,Phoenix,Arizona,1660272.0,1200.0,21907.000000,1.864420e+06,1
6,Philadelphia,Pennsylvania,1584138.0,4511.0,22874.000000,2.816850e+06,1
7,San Antonio,Texas,1532233.0,1250.0,18518.000000,9.239900e+05,1
8,San Diego,California,1425976.0,1670.0,22926.000000,9.877600e+05,0
9,Dallas,Texas,1345047.0,1493.0,23616.000000,2.622070e+06,1


In [558]:
df_data.NBA.value_counts()

# set outlier manually (Golden State = San Francisco)
cols = df_data['City']=='San Francisco'
df_data.loc[cols,'NBA']=1
df_data

,City,State,Population,Density,PCI,TV,NBA
Rank,,,,,,,
1,New York,New York,8398748.0,10933.0,24581.000000,7.100300e+06,1
2,Los Angeles,California,3990456.0,3276.0,21170.000000,5.276600e+06,1
3,Chicago,Illinois,2705994.0,4600.0,21435.500000,3.251370e+06,1
4,Houston,Texas,2325502.0,1395.0,21701.000000,2.423360e+06,1
5,Phoenix,Arizona,1660272.0,1200.0,21907.000000,1.864420e+06,1
6,Philadelphia,Pennsylvania,1584138.0,4511.0,22874.000000,2.816850e+06,1
7,San Antonio,Texas,1532233.0,1250.0,18518.000000,9.239900e+05,1
8,San Diego,California,1425976.0,1670.0,22926.000000,9.877600e+05,0
9,Dallas,Texas,1345047.0,1493.0,23616.000000,2.622070e+06,1


In [559]:
# take only the top 100 largest cities to both reduce data size and the realization that small cities 
# won't be able support an NBA franchse
city_data = df_data.iloc[0:100]
city_data

,City,State,Population,Density,PCI,TV,NBA
Rank,,,,,,,
1,New York,New York,8398748.0,10933.0,24581.0,7.100300e+06,1
2,Los Angeles,California,3990456.0,3276.0,21170.0,5.276600e+06,1
3,Chicago,Illinois,2705994.0,4600.0,21435.5,3.251370e+06,1
4,Houston,Texas,2325502.0,1395.0,21701.0,2.423360e+06,1
5,Phoenix,Arizona,1660272.0,1200.0,21907.0,1.864420e+06,1
6,Philadelphia,Pennsylvania,1584138.0,4511.0,22874.0,2.816850e+06,1
7,San Antonio,Texas,1532233.0,1250.0,18518.0,9.239900e+05,1
8,San Diego,California,1425976.0,1670.0,22926.0,9.877600e+05,0
9,Dallas,Texas,1345047.0,1493.0,23616.0,2.622070e+06,1


### Get the City Longitude and Latitude

In [560]:
!{sys.executable} -m pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

  Using cached https://files.pythonhosted.org/packages/80/93/d384479da0ead712bdaf697a8399c13a9a89bd856ada5a27d462fb45e47b/geopy-1.20.0-py2.py3-none-any.whl


In [561]:
# create a function to add the latitude and longitude for each city
def get_long_lat(city,state,long=1):
    address = str(city)+', '+str(state)
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    if long==1:
        return longitude
    else:
        return latitude
   
# get_long_lat('San Jose','California',0)

In [562]:
# add the long and lat to the dataframe
city_data['Long'] = city_data.apply(lambda x: get_long_lat(x['City'],x['State'],1),axis=1)
city_data['Lat'] = city_data.apply(lambda x: get_long_lat(x['City'],x['State'],0),axis=1)
city_data

/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,State,Population,Density,PCI,TV,NBA,Long,Lat
Rank,,,,,,,,,
1,New York,New York,8398748.0,10933.0,24581.0,7.100300e+06,1,-74.006015,40.712728
2,Los Angeles,California,3990456.0,3276.0,21170.0,5.276600e+06,1,-118.242767,34.053691
3,Chicago,Illinois,2705994.0,4600.0,21435.5,3.251370e+06,1,-87.624421,41.875562
4,Houston,Texas,2325502.0,1395.0,21701.0,2.423360e+06,1,-95.367697,29.758938
5,Phoenix,Arizona,1660272.0,1200.0,21907.0,1.864420e+06,1,-112.077346,33.448587
6,Philadelphia,Pennsylvania,1584138.0,4511.0,22874.0,2.816850e+06,1,-75.163526,39.952724
7,San Antonio,Texas,1532233.0,1250.0,18518.0,9.239900e+05,1,-98.495141,29.424600
8,San Diego,California,1425976.0,1670.0,22926.0,9.877600e+05,0,-117.162771,32.717421
9,Dallas,Texas,1345047.0,1493.0,23616.0,2.622070e+06,1,-96.796856,32.776272


Let's now create a map of the US with the city data plotted 

In [563]:
# create map of US
latitude = 42.8777
longitude = -97.38
map_us = folium.Map(location=[latitude, longitude], zoom_start=4)

# add city markers to map
for lat, lng, label in zip(city_data['Lat'], city_data['Long'], city_data['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_us)  
    
map_us


### Adding the FourSquare Data

In [564]:
CLIENT_ID = 'X1RDABIJGUOLEDZZHTFOOIME4KPAMKJPOXRRZZLYWJD2NBHT' # your Foursquare ID
CLIENT_SECRET = 'JJ35MDOBWT40B44T4UD3WMMU4F0RSU5TFO2F5AYQMJCMLYG4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X1RDABIJGUOLEDZZHTFOOIME4KPAMKJPOXRRZZLYWJD2NBHT
CLIENT_SECRET:JJ35MDOBWT40B44T4UD3WMMU4F0RSU5TFO2F5AYQMJCMLYG4


Let's write a function to get nearby venues for each city in our list.
We will consider venues within 5 km of the city center, limiting our response to 1000

In [495]:
radius=5000
LIMIT=100
# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&intent=browse&radius={}&limit={}'.format(
     CLIENT_ID, 
     CLIENT_SECRET, 
     VERSION, 
     latitude, 
     longitude, 
     radius, 
     LIMIT)
            
# make the GET request
results = requests.get(url).json() 
results["response"]['venues'][0]['categories']


[{'id': '5032897c91d4c4b30a586d69',
  'name': 'Pet Service',
  'pluralName': 'Pet Services',
  'shortName': 'Pet Service',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_',
   'suffix': '.png'},
  'primary': True}]

In [565]:
def getCityVenues(cities, states, latitudes, longitudes, radius=1000, LIMIT=100):
    
    venues_list=[]
    for city, state, lat, lng in zip(cities, states, latitudes, longitudes):
        print(city, state)
        
        # will repeat this request 5 times per city to get enough location data
        # once at the base lat/long and the one at 0.1 increments at each
        lats = [lat,lat,lat,lat+0.2,lat-0.2]
        lngs = [lng,lng+0.2,lng-0.2,lng,lng]
        for la, ln in zip(lats,lngs):
            print(la,ln)
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                la, 
                ln, 
                radius, 
                LIMIT)
            
            try:
                # make the GET request
                results = requests.get(url).json()["response"]['groups'][0]['items']
        
                # return only relevant information for each nearby venue
                venues_list.append([(
                    city,
                    state, 
                    lat, 
                    lng, 
                    v['venue']['name'], 
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng'],  
                    v['venue']['categories'][0]['name']) for v in results])
            except Exception as e:
                print(e)
                
    city_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    print(city_venues)
    city_venues.columns = ['City', 
                           'State',
                           'Latitude', 
                           'Longitude', 
                           'Venue', 
                           'Venue Lat', 
                           'Venue Long', 
                           'Venue Category']
    
    return(city_venues)

In [566]:
#small = city_data.iloc[0:1]
# now let's collect the city venues
city_venues = getCityVenues(cities=city_data['City'], states=city_data['State'],
                                   latitudes=city_data['Lat'],
                                   longitudes=city_data['Long']
                                  )

#city_venues = getCityVenues(cities=small['City'], states=small['State'],
#                                   latitudes=small['Lat'],
#                                   longitudes=small['Long']
#                                  )

city_venues

New York New York
40.7127281 -74.0060152
40.7127281 -73.80601519999999
40.7127281 -74.2060152
40.9127281 -74.0060152
40.5127281 -74.0060152
Los Angeles California
34.0536909 -118.2427666
34.0536909 -118.0427666
34.0536909 -118.4427666
34.2536909 -118.2427666
33.8536909 -118.2427666
Chicago Illinois
41.8755616 -87.6244212
41.8755616 -87.4244212
41.8755616 -87.8244212
42.0755616 -87.6244212
41.675561599999995 -87.6244212
Houston Texas
29.7589382 -95.3676974
'groups'
29.7589382 -95.1676974
29.7589382 -95.5676974
29.9589382 -95.3676974
29.5589382 -95.3676974
Phoenix Arizona
33.4485866 -112.0773456
33.4485866 -111.8773456
33.4485866 -112.2773456
33.6485866 -112.0773456
33.248586599999996 -112.0773456
Philadelphia Pennsylvania
39.9527237 -75.1635262
39.9527237 -74.9635262
39.9527237 -75.36352620000001
40.1527237 -75.1635262
39.7527237 -75.1635262
San Antonio Texas
29.4246002 -98.4951405
29.4246002 -98.2951405
29.4246002 -98.69514050000001
29.6246002 -98.4951405
29.2246002 -98.4951405
San Die

33.7494951 -118.0732213
33.9494951 -117.8732213
33.549495099999994 -117.8732213
Riverside California
33.9533546 -117.3961623
33.9533546 -117.1961623
33.9533546 -117.5961623
34.1533546 -117.3961623
33.753354599999994 -117.3961623
Corpus Christi Texas
27.8002542 -97.3955744
27.8002542 -97.1955744
27.8002542 -97.5955744
28.0002542 -97.3955744
27.600254200000002 -97.3955744
Lexington Kentucky
38.0464066 -84.4970393
38.0464066 -84.2970393
38.0464066 -84.6970393
38.2464066 -84.4970393
37.846406599999995 -84.4970393
Stockton California
37.9577016 -121.2907796
37.9577016 -121.09077959999999
37.9577016 -121.4907796
38.1577016 -121.2907796
37.7577016 -121.2907796
Henderson Nevada
36.0391456 -114.9819235
36.0391456 -114.78192349999999
36.0391456 -115.1819235
36.2391456 -114.9819235
35.839145599999995 -114.9819235
Saint Paul Minnesota
44.9504037 -93.1015026
44.9504037 -92.9015026
44.9504037 -93.3015026
45.150403700000005 -93.1015026
44.7504037 -93.1015026
St. Louis Missouri
38.6268039 -90.1994097


,City,State,Latitude,Longitude,Venue,Venue Lat,Venue Long,Venue Category
0,New York,New York,40.712728,-74.006015,The Bar Room at Temple Court,40.711448,-74.006802,Hotel Bar
1,New York,New York,40.712728,-74.006015,The Beekman - A Thompson Hotel,40.711173,-74.006702,Hotel
2,New York,New York,40.712728,-74.006015,Alba Dry Cleaner & Tailor,40.711434,-74.006272,Laundry Service
3,New York,New York,40.712728,-74.006015,The Wooly Daily,40.712137,-74.008395,Coffee Shop
4,New York,New York,40.712728,-74.006015,City Hall Park,40.712415,-74.006724,Park
5,New York,New York,40.712728,-74.006015,Gibney Dance Center Downtown,40.713923,-74.005661,Dance Studio
6,New York,New York,40.712728,-74.006015,Augustine,40.711310,-74.006660,French Restaurant
7,New York,New York,40.712728,-74.006015,The Class by Taryn Toomey,40.712753,-74.008734,Gym / Fitness Center
8,New York,New York,40.712728,-74.006015,Takahachi Bakery,40.713653,-74.008804,Bakery
9,New York,New York,40.712728,-74.006015,Four Seasons Hotel New York Downtown,40.712612,-74.009380,Hotel


In [567]:
city_venues

,City,State,Latitude,Longitude,Venue,Venue Lat,Venue Long,Venue Category
0,New York,New York,40.712728,-74.006015,The Bar Room at Temple Court,40.711448,-74.006802,Hotel Bar
1,New York,New York,40.712728,-74.006015,The Beekman - A Thompson Hotel,40.711173,-74.006702,Hotel
2,New York,New York,40.712728,-74.006015,Alba Dry Cleaner & Tailor,40.711434,-74.006272,Laundry Service
3,New York,New York,40.712728,-74.006015,The Wooly Daily,40.712137,-74.008395,Coffee Shop
4,New York,New York,40.712728,-74.006015,City Hall Park,40.712415,-74.006724,Park
5,New York,New York,40.712728,-74.006015,Gibney Dance Center Downtown,40.713923,-74.005661,Dance Studio
6,New York,New York,40.712728,-74.006015,Augustine,40.711310,-74.006660,French Restaurant
7,New York,New York,40.712728,-74.006015,The Class by Taryn Toomey,40.712753,-74.008734,Gym / Fitness Center
8,New York,New York,40.712728,-74.006015,Takahachi Bakery,40.713653,-74.008804,Bakery
9,New York,New York,40.712728,-74.006015,Four Seasons Hotel New York Downtown,40.712612,-74.009380,Hotel


Next, we need to extract and one-hot the locale information

In [568]:
# define a function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [572]:
city_venues['Venue Category'].value_counts()

Coffee Shop                                 381
Hotel                                       319
American Restaurant                         250
Pizza Place                                 246
Mexican Restaurant                          238
Bar                                         233
Sandwich Place                              233
Park                                        190
Fast Food Restaurant                        136
Italian Restaurant                          131
Café                                        111
Restaurant                                   99
Cocktail Bar                                 93
Seafood Restaurant                           93
Sushi Restaurant                             92
Theater                                      90
Steakhouse                                   89
Pharmacy                                     84
Burger Joint                                 83
Gym                                          82
Bakery                                  

In [576]:
# one hot encoding
cities_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add city and state information back in
cities_onehot['City'] = city_venues['City'] 
cities_onehot['State'] = city_venues['State'] 
cities_onehot.head()

,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Carpet Store,Casino,Cave,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Quad,College Rec Center,College Residence Hall,College Soccer Field,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Country Dance Club,Coworking Space,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nation

In [582]:
# move city and state back to the front
fixed_columns = [cities_onehot.columns[-2]] + [cities_onehot.columns[-1]] + list(cities_onehot.columns[:-2])
cities_onehot = cities_onehot[fixed_columns]

cities_onehot.head()

,City,State,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Carpet Store,Casino,Cave,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Quad,College Rec Center,College Residence Hall,College Soccer Field,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Country Dance Club,Coworking Space,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail S

Grouping the data by city

In [586]:
grouped_city_venues = cities_onehot.groupby('City').mean().reset_index()
grouped_city_venues.head()

,City,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Carpet Store,Casino,Cave,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Quad,College Rec Center,College Residence Hall,College Soccer Field,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Country Dance Club,Coworking Space,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,N

In [589]:
grouped_city_venues.shape

(64, 435)